In [15]:
import Bens_forecasting_utils as fc
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet
%matplotlib ipympl
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from pandas.tseries.offsets import MonthEnd
plt.rcParams['figure.figsize'] = [10, 5] # make default plot size bigger
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

pd.options.display.max_rows = None

In [16]:
parameters = dict(
                sheet_id = '1cbW6p8rdMhq9XD5WL-QJNO1UBRQRTcKQBoh6g1QfEv8',
                copied_marketing_file = 'actual_budget_2025!A1:Y9',
                tnc_mapping = 'actual_budget_2025!A22:D57')

#read the correct mau table under first tab in marketing file
marketing_df = fc.import_gsheet_to_df(parameters["sheet_id"], parameters["copied_marketing_file"]) 
marketing_df.head()


,Cum. Net MAU,Jan 24,Feb 24,Mar 24,Apr 24,May 24,Jun 24,Jul 24,Aug 24,Sep 24,...,Mar 25,Apr 25,May 25,Jun 25,Jul 25,Aug 25,Sep 25,Oct 25,Nov 25,Dec 25
1,Germany,"938,923","948,289","962,084","976,868","985,266","999,052","1,015,667","1,025,103","1,048,425",...,"1,177,719","1,200,143","1,224,672","1,249,421","1,275,007","1,302,911","1,336,537","1,371,818","1,403,034","1,429,577"
2,Austria,"92,804","93,592","94,898","96,202","97,492","98,712","100,853","101,771","104,294",...,"118,936","121,912","124,980","127,844","130,703","133,500","136,863","139,999","142,582","144,815"
3,France,"402,334","402,655","403,002","405,338","405,793","406,600","413,274","418,117","428,581",...,"484,404","496,014","507,770","518,351","530,132","543,210","558,813","576,143","592,228","603,553"
4,Italy,"221,169","218,012","216,048","213,864","211,890","210,061","211,772","211,710","212,814",...,"210,992","210,114","209,220","211,361","213,989","218,318","223,955","227,275","230,125","231,764"
5,Spain,"242,213","247,998","254,786","257,942","262,081","269,578","278,198","282,133","285,685",...,"332,118","342,416","353,348","362,847","373,065","383,571","397,672","413,827","428,475","440,860"


In [17]:
# perform table transformations
marketing_df = marketing_df.T  
marketing_df = marketing_df.reset_index()
marketing_df.columns = ["Cum. Net MAU"] + list(marketing_df.iloc[0, 1:])  
marketing_df = marketing_df[1:].reset_index(drop=True)
marketing_df.rename(columns={"Cum. Net MAU": "Months"}, inplace=True)
marketing_df.head()

,Months,Germany,Austria,France,Italy,Spain,Greater Europe,Non-Euro,Total
0,Jan 24,"938,923","92,804","402,334","221,169","242,213","193,827","27,107","2,118,377"
1,Feb 24,"948,289","93,592","402,655","218,012","247,998","193,671","27,310","2,131,527"
2,Mar 24,"962,084","94,898","403,002","216,048","254,786","197,417","27,305","2,155,540"
3,Apr 24,"976,868","96,202","405,338","213,864","257,942","196,750","27,429","2,174,393"
4,May 24,"985,266","97,492","405,793","211,890","262,081","197,474","27,604","2,187,600"


In [18]:
# add weeks to interpolate later
def add_weeks(marketing_df):
    week_rows = []
    
    for i, row in marketing_df.iterrows():
        month = row["Months"]  # month 
        month_number = f"{i+1:02d}" 

        week_rows.append(row)

        for week in range(1, 5):
            week_row = row.copy()
            week_row["Months"] = f"W{week:02d}-{month_number}"  # Format: "W01-01", "W02-01", etc.

            for col in marketing_df.columns[1:]:  
                week_row[col] = None  
            
            week_rows.append(week_row)
    
    weeks_df = pd.DataFrame(week_rows)
    
    return weeks_df

marketing_df = add_weeks(marketing_df)
marketing_df.head()

,Months,Germany,Austria,France,Italy,Spain,Greater Europe,Non-Euro,Total
0,Jan 24,"938,923","92,804","402,334","221,169","242,213","193,827","27,107","2,118,377"
0,W01-01,None,None,None,None,None,None,None,None
0,W02-01,None,None,None,None,None,None,None,None
0,W03-01,None,None,None,None,None,None,None,None
0,W04-01,None,None,None,None,None,None,None,None


In [19]:
# import numpy as np
# import pandas as pd

# def interpolate_weeks(df):
#     result = df.copy()

#     # Convert numerical columns to float
#     for col in result.columns[1:]:
#         result[col] = pd.to_numeric(result[col].astype(str).str.replace(',', ''), errors='coerce')

#     interpolated_rows = []

#     # Static weights for each week in a month (total should sum to 1)
#     static_weights = np.array([0.2, 0.25, 0.21, 0.34])

#     for i in range(len(result)):
#         month_row = result.iloc[i]
#         month_name = month_row['Months']

#         if ' ' in month_name:  # Identify monthly total rows
#             month_total = month_row[1:].values  # Extract numerical values

#             # Distribute the month total across the weeks using the static weights
#             week_values = (month_total[:, None] * static_weights).T  # Scale by static weights

#             # Generate interpolated rows for each week
#             for w in range(4):
#                 week_label = f"W{w+1}- {month_name}"

#                 # Create new row
#                 week_row = [week_label] + list(week_values[w])
#                 interpolated_rows.append(week_row)

#     # Create final DataFrame with interpolated weeks
#     weekly_columns = ['Months'] + list(result.columns[1:])
#     interpolated_df = pd.DataFrame(interpolated_rows, columns=weekly_columns)

#     return interpolated_df


In [20]:
def interpolate_weeks(df):
    result = df.copy()

    for col in result.columns[1:]:
        result[col] = pd.to_numeric(result[col].str.replace(',', ''), errors='coerce')

    weights = np.array([0.24, 0.26, 0.27, 0.23]) 

    weekly_rows = []

    for i in range(0, len(result), 5):
        month_row = result.iloc[i]
        month_name = month_row['Months']
        year_suffix = month_name.split()[-1][-2:] 
        month_total = month_row[1:].values  # exclude 'Months' column

        for w in range(4):
            week_name = f"W{w+1}-{month_name[:4]}-{year_suffix}"  # example W01-Jan-24
            weekly_values = month_total * weights[w] 
            weekly_rows.append([week_name] + list(weekly_values))

    weekly_columns = ['Months'] + list(result.columns[1:])
    weekly_df = pd.DataFrame(weekly_rows, columns=weekly_columns)

    return weekly_df

In [21]:
# apply interpolation
interpolated_marketing_df = interpolate_weeks(marketing_df)
interpolated_marketing_df.head(10)

,Months,Germany,Austria,France,Italy,Spain,Greater Europe,Non-Euro,Total
0,W1- Jan-24,225341.52,22272.96,96560.16,53080.56,58131.12,46518.48,6505.68,508410.48
1,W2- Jan-24,244119.98,24129.04,104606.84,57503.94,62975.38,50395.02,7047.82,550778.02
2,W3- Jan-24,253509.21,25057.08,108630.18,59715.63,65397.51,52333.29,7318.89,571961.79
3,W4- Jan-24,215952.29,21344.92,92536.82,50868.87,55708.99,44580.21,6234.61,487226.71
4,W1- Feb-24,227589.36,22462.08,96637.20,52322.88,59519.52,46481.04,6554.40,511566.48
5,W2- Feb-24,246555.14,24333.92,104690.30,56683.12,64479.48,50354.46,7100.60,554197.02
6,W3- Feb-24,256038.03,25269.84,108716.85,58863.24,66959.46,52291.17,7373.70,575512.29
7,W4- Feb-24,218106.47,21526.16,92610.65,50142.76,57039.54,44544.33,6281.30,490251.21
8,W1- Mar-24,230900.16,22775.52,96720.48,51851.52,61148.64,47380.08,6553.20,517329.60
9,W2- Mar-24,250141.84,24673.48,104780.52,56172.48,66244.36,51328.42,7099.30,560440.40


In [22]:
def extrapolate_weekly(df, months_ahead=24):
    df = df.copy()
    
    weights = np.array([0.24, 0.26, 0.27, 0.23])
    
    def parse_date(x):
        parts = x.split('-')
        if len(parts) == 3:  
            _, month, year = parts
            return pd.to_datetime(f'20{year}-{month}-01')
        else:  # format W1- Jan 24
            week = parts[0]
            month_year = ' '.join(parts[1:]).strip()
            month, year = month_year.split()
            return pd.to_datetime(f'20{year}-{month}-01')
    
    df['ds'] = df['Months'].apply(parse_date)
    
    forecasts = {}
    for col in df.columns:
        if col in ['ds', 'Months']:
            continue
            
        monthly_values = pd.to_numeric(df[col].astype(str).str.replace(',', '').str.replace('e+', 'e', regex=False), 
                                     errors='coerce')
        dates = df['ds']
        
        all_weeks = []
        all_values = []
        
        for i in range(len(monthly_values)-1):
            start_val = monthly_values.iloc[i]
            end_val = monthly_values.iloc[i+1]
            start_date = dates.iloc[i]
            end_date = dates.iloc[i+1]
            
            # 4 weeks between months
            week_dates = pd.date_range(start_date, end_date, periods=5)[:-1]
            
            week_values = start_val * weights
            
            all_weeks.extend(week_dates)
            all_values.extend(week_values)
        
        last_weeks = pd.date_range(dates.iloc[-1], periods=4, freq='W')
        last_values = monthly_values.iloc[-1] * weights
        all_weeks.extend(last_weeks)
        all_values.extend(last_values)
        
        # Prophet df
        prophet_df = pd.DataFrame({
            'ds': all_weeks,
            'y': all_values})
        
        model = Prophet(yearly_seasonality=True)
        model.fit(prophet_df)
        
        future = model.make_future_dataframe(periods=months_ahead * 4, freq='W')
        forecast = model.predict(future)
        forecasts[col] = np.maximum(forecast['yhat'].values[-(months_ahead * 4):], 0)
        
    #future dates
    last_date = df['ds'].max()
    future_dates = pd.date_range(start=last_date, periods=months_ahead * 4, freq='W')
    
    future_labels = []
    for date in future_dates:
        week = (date.day - 1) // 7 + 1
        future_labels.append(f'W{week}-{date.strftime("%b-%y")}')  # Removed space after hyphen
    
    return pd.DataFrame({
        'Months': future_labels, **{col: forecasts[col] for col in forecasts}})

In [23]:
# apply extrapolated function
forecasted_df = extrapolate_weekly(interpolated_marketing_df, months_ahead=27)
forecasted_df.tail()

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /var/folders/ld/t4l42g9d2kndcwfk5qh21smc0000gp/T/tmpthcw4s9w/gub1aw0h.json
DEBUG:cmdstanpy:input tempfile: /var/folders/ld/t4l42g9d2kndcwfk5qh21smc0000gp/T/tmpthcw4s9w/8_wdfcio.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/Users/krisi.afezolli/Library/Python/3.13/lib/python/site-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=16239', 'data', 'file=/var/folders/ld/t4l42g9d2kndcwfk5qh21smc0000gp/T/tmpthcw4s9w/gub1aw0h.json', 'init=/var/folders/ld/t4l42g9d2kndcwfk5qh21smc0000gp/T/tmpthcw4s9w/8_wdfcio.json', 'output', 'file=/var/folders/ld/t4l42g9d2kndcwfk5qh21smc0000gp/T/tmpthcw4s9w/prophet_modelcaxg2rds/prophet_model-20250210091718.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
09:17:18 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start pro

,Months,Germany,Austria,France,Italy,Spain,Greater Europe,Non-Euro,Total
103,W4-Nov-27,122391.317072,12841.472759,54456.666041,15389.617209,42063.467764,25106.900412,2855.495004,274213.818168
104,W1-Dec-27,122723.100109,12913.932686,54743.794578,15309.440817,42110.706794,25029.210158,2877.795161,274741.363961
105,W2-Dec-27,123350.237645,12998.211306,55094.726407,15291.338780,42330.045382,25092.333166,2898.903292,276045.702958
106,W3-Dec-27,124093.128120,13075.030968,55437.638004,15354.503410,42636.130946,25254.929994,2916.932762,277772.969333
107,W4-Dec-27,124776.482590,13140.699665,55729.646476,15460.799074,42916.537159,25419.076628,2934.995960,279429.519564


In [24]:
# concatenate both df: interpolated one + extraplated one
df = pd.concat([interpolated_marketing_df, forecasted_df.loc[4:]], ignore_index=True)
df.tail()

,Months,Germany,Austria,France,Italy,Spain,Greater Europe,Non-Euro,Total
195,W4-Nov-27,122391.317072,12841.472759,54456.666041,15389.617209,42063.467764,25106.900412,2855.495004,274213.818168
196,W1-Dec-27,122723.100109,12913.932686,54743.794578,15309.440817,42110.706794,25029.210158,2877.795161,274741.363961
197,W2-Dec-27,123350.237645,12998.211306,55094.726407,15291.338780,42330.045382,25092.333166,2898.903292,276045.702958
198,W3-Dec-27,124093.128120,13075.030968,55437.638004,15354.503410,42636.130946,25254.929994,2916.932762,277772.969333
199,W4-Dec-27,124776.482590,13140.699665,55729.646476,15460.799074,42916.537159,25419.076628,2934.995960,279429.519564


### Importing tnc_mapping language table

In [25]:
tnc_df = fc.import_gsheet_to_df(parameters["sheet_id"], parameters["tnc_mapping"]) 
tnc_df.head()

,Lookup,TnC Country,Language,Language % Split
1,AUT::de,AUT,de,80.06%
2,AUT::en,AUT,en,18.12%
3,AUT::es,AUT,es,1.08%
4,AUT::fr,AUT,fr,0.22%
5,AUT::it,AUT,it,0.48%


In [26]:
tnc_df['Language % Split'] = tnc_df['Language % Split'].astype(str).str.rstrip('%').astype(float)

# Mapping of country codes to column names in df
country_code_mapping = {
    'AUT': 'Austria',
    'DEU': 'Germany',
    'ITA': 'Italy',
    'FRA': 'France',
    'ESP': 'Spain',
    'GrE': 'Greater Europe',
    'NEuro': 'Non-Euro'}

def get_year_from_month(month_str):
    # Extract the year part after the month (e.g., "Jan 24" -> "24")
    year_str = month_str.split()[-1]
    return int(year_str) + 2000  # Convert "24" to 2024

expanded_rows = []

for index, row in df.iterrows():
    month_part = row['Months'].split('-')[1:]  
    month_str = ' '.join(month_part).strip()
    year = get_year_from_month(month_str)
    
    for country_code, country_name in country_code_mapping.items():
        if country_name not in row:
            continue
        
        country_tnc_df = tnc_df[tnc_df['TnC Country'] == country_code]
        
        for _, lang_row in country_tnc_df.iterrows():
            lang_split = lang_row['Language % Split'] / 100
            
            new_row = {
                'Calendar Week': row['Months'],
                'Country': country_name,
                'Language': lang_row['Language'],
                'Type': 'interpolation' if 2024 <= year <= 2025 else 'extrapolation',
                'MAU': row[country_name] * lang_split
            }
            
            expanded_rows.append(new_row)

expanded_df = pd.DataFrame(expanded_rows)

In [27]:
expanded_df_filtered_sorted = expanded_df.sort_values(by=['Language', 'Country'])

expanded_df_filtered_sorted['% Change'] = expanded_df_filtered_sorted.groupby(['Country', 'Language'])['MAU'].pct_change() * 100
expanded_df_filtered_sorted['% Change'] = expanded_df_filtered_sorted['% Change'].apply(lambda x: f"{x:.2f}%")

expanded_df_filtered_sorted[['Calendar Week', 'Country', 'Language', 'MAU','% Change']].head(10)



,Calendar Week,Country,Language,MAU,% Change
0,W1- Jan-24,Austria,de,17831.731776,nan%
35,W2- Jan-24,Austria,de,19317.709424,8.33%
70,W3- Jan-24,Austria,de,20060.698248,3.85%
105,W4- Jan-24,Austria,de,17088.742952,-14.81%
140,W1- Feb-24,Austria,de,17983.141248,5.23%
175,W2- Feb-24,Austria,de,19481.736352,8.33%
210,W3- Feb-24,Austria,de,20231.033904,3.85%
245,W4- Feb-24,Austria,de,17233.843696,-14.81%
280,W1- Mar-24,Austria,de,18234.081312,5.80%
315,W2- Mar-24,Austria,de,19753.588088,8.33%


### Export expanded_df to copied_marketing gsheet we created first thing

In [28]:
gsheet_export_params = dict(
    df = expanded_df_filtered_sorted, 
    gsheet_id = parameters["sheet_id"],
    gsheet_tab_name = 'Final',
    include_df_headers = True,
    tab_colour = (0.0, 0.0, 0.0))

fc.export_df_to_google_sheet(**gsheet_export_params)